In [1]:
import tensorflow as tf
import numpy as np
import src

In [2]:
problem = src.SortingProblem()

In [ ]:
tf.reset_default_graph()
network = src.PointerNetwork('cfg')

seqLen = network.maxTimeSteps
batchSize = network.batchSize

In [ ]:
trainingParams = {}
trainingParams['batchSize'] = batchSize
trainingParams['length'] = seqLen

losses = []

saver = tf.train.Saver()
with tf.Session() as sess:
    # initialize variables
    sess.run(tf.global_variables_initializer())
    
    # write out log for graph visualization
    writer = tf.summary.FileWriter('checkpoints/', sess.graph)

    for batch in range(100):
        # make current batch of training data
        inputs = problem.makeBatch(trainingParams)
        trainingInputs, targets = problem.makeTargets(inputs)
        
        # apply one training step
        _, loss = sess.run([network.trainOp, network.loss], 
                           feed_dict={network.train: True,
                                      network.rawInputs: inputs,
                                      network.targets: targets,
                                      network.targetInputs: trainingInputs[:,:-1,:]})
        
        # keep track of the loss values
        losses.append(loss)
        
        if batch % 100 == 0 or batch == 99:
            
            # generate a test batch and output its accuracy and also a sample
            print('Batch: ' + str(batch))
            newInputs = problem.makeBatch(trainingParams)
            output = sess.run(network.decoderOutputs, feed_dict={network.train: False,
                                                                 network.rawInputs: newInputs})
            print(newInputs[0, :, :].T[0])
            print(newInputs[0, :, :].T[0][np.array(output).T[0]])
            print(problem.accuracy(solution=np.array(output).T, batchArray=newInputs))
            
            # save checkpoint
            if batch > 0:
                saver.save(sess, 'checkpoints/SortingPointerNetwork', global_step=network.globalStep)

Batch: 0
[ 0.0104827   0.67190734  0.1791262   0.91124201  0.4849316   0.99031874
  0.36407581  0.87275112  0.71377798  0.7469021   0.12385637  0.16245243
  0.54418423  0.35802675  0.04387939]
[ 0.04387939  0.0104827   0.1791262   0.99031874  0.91124201  0.87275112
  0.67190734  0.7469021   0.71377798  0.54418423  0.4849316   0.35802675
  0.36407581  0.16245243  0.12385637]
0.0


In [ ]:
import matplotlib.pyplot as plt

plt.semilogy(losses)
plt.show()